# CommaSeparated Parser

In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain.prompts import PromptTemplate, SystemMessagePromptTemplate, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_openai import OpenAI
from langchain.chat_models import ChatOpenAI

In [3]:
from langchain.output_parsers import CommaSeparatedListOutputParser

In [4]:
out_parser = CommaSeparatedListOutputParser()

In [6]:
format_instruction = out_parser.get_format_instructions()

In [8]:
format_instruction

'Your response should be a list of comma separated values, eg: `foo, bar, baz` or `foo,bar,baz`'

In [7]:
out_parser.parse("a,b,c,d")

['a', 'b', 'c', 'd']

In [9]:
human_template = """Please suggest 5 must-watch {genre} movies from different decades.
{format_instructions}"""

In [17]:
# Set up the prompt structure
human_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages([human_prompt])

# Format the prompt with specific genre
request = chat_prompt.format_prompt(
    genre="comedy",
    format_instructions=out_parser.get_format_instructions()
).to_messages()

model = OpenAI()

# Get the response from the model
result = model.invoke(request)

In [18]:
result

'\n\n1. Monty Python and the Holy Grail (1975)\n2. Airplane! (1980)\n3. Groundhog Day (1993)\n4. The Big Lebowski (1998)\n5. Bridesmaids (2011)'

In [19]:
movies = out_parser.parse(result)

In [20]:
movies

['1. Monty Python and the Holy Grail (1975)',
 '2. Airplane! (1980)',
 '3. Groundhog Day (1993)',
 '4. The Big Lebowski (1998)',
 '5. Bridesmaids (2011)']

# JSON parser

In [21]:
from langchain_core.output_parsers import JsonOutputParser
from langchain.prompts import HumanMessagePromptTemplate, ChatPromptTemplate

# Define a simple JSON schema for 5 movie titles
schema = {
    "type": "array",
    "items": {"type": "string"},
    "minItems": 5,
    "maxItems": 5
}

# Set up the JSON parser and prompt
parser = JsonOutputParser(json_schema=schema)
template = """Suggest 5 must-watch {genre} movies from different decades.
{format_instructions}"""

chat_prompt = ChatPromptTemplate.from_messages([
    HumanMessagePromptTemplate.from_template(template)
])

# Prepare the request, get response, and parse JSON
messages = chat_prompt.format_prompt(
    genre="sci-fi", 
    format_instructions=parser.get_format_instructions()
).to_messages()

result = model.invoke(messages)
print(result)
movies = parser.parse(result)
print(movies)



{
  "1950s": "The Day the Earth Stood Still",
  "1960s": "2001: A Space Odyssey",
  "1970s": "Star Wars",
  "1980s": "Blade Runner",
  "1990s": "The Matrix"
}
{'1950s': 'The Day the Earth Stood Still', '1960s': '2001: A Space Odyssey', '1970s': 'Star Wars', '1980s': 'Blade Runner', '1990s': 'The Matrix'}


In [22]:
parser.get_format_instructions()

'Return a JSON object.'

In [23]:
type(movies)

dict

In [24]:
movies['1950s']

'The Day the Earth Stood Still'

# XML parser

In [26]:
import xml.etree.ElementTree as ET
from langchain.prompts import HumanMessagePromptTemplate, ChatPromptTemplate

# Define an XML-specific prompt template instructing the model to output XML
template = """Suggest 5 must-watch {genre} movies from different decades.
Please format the response as XML with a root element <movies> and each movie as a <movie> element containing <title> and <decade> children."""

chat_prompt = ChatPromptTemplate.from_messages([
    HumanMessagePromptTemplate.from_template(template)
])

# Prepare the request with the desired genre
messages = chat_prompt.format_prompt(genre="sci-fi").to_messages()

# Invoke the model
result = model.invoke(messages)
xml_response = result
print("XML Response from model:")
print(xml_response)

# Define a function to parse the XML response
def parse_xml_movies(xml_string):
    root = ET.fromstring(xml_string)
    movies = []
    for movie_elem in root.findall('movie'):
        title = movie_elem.find('title').text if movie_elem.find('title') is not None else "N/A"
        decade = movie_elem.find('decade').text if movie_elem.find('decade') is not None else "N/A"
        movies.append({'title': title, 'decade': decade})
    return movies

# Parse the XML into a Python list of dictionaries
movies = parse_xml_movies(xml_response)

# Print the recommended movies
print("\nRecommended Sci-Fi Movies:")
for movie in movies:
    print(f"- {movie['title']} ({movie['decade']})")

XML Response from model:


<movies>
  <movie>
    <title>2001: A Space Odyssey</title>
    <decade>1960s</decade>
  </movie>
  <movie>
    <title>Blade Runner</title>
    <decade>1980s</decade>
  </movie>
  <movie>
    <title>The Matrix</title>
    <decade>1990s</decade>
  </movie>
  <movie>
    <title>Inception</title>
    <decade>2010s</decade>
  </movie>
  <movie>
    <title>Arrival</title>
    <decade>2020s</decade>
  </movie>
</movies>

Recommended Sci-Fi Movies:
- 2001: A Space Odyssey (1960s)
- Blade Runner (1980s)
- The Matrix (1990s)
- Inception (2010s)
- Arrival (2020s)


# datetime Parser

In [30]:
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_openai import ChatOpenAI
from langchain.output_parsers import DatetimeOutputParser
from datetime import datetime
import pytz

# Initialize model and parser
model = ChatOpenAI()
datetime_parser = DatetimeOutputParser()

# Create prompt templates
current_date = datetime.now(pytz.UTC).strftime("%Y-%m-%d")
system_prompt = SystemMessagePromptTemplate.from_template(
    f"You are a helpful assistant that converts text descriptions of dates and times into ISO format datetime strings.\n"
    f"Always return the datetime in the exact format: YYYY-MM-DDThh:mm:ss.000000Z\n"
    f"For relative dates, use the current date ({current_date}) as the reference point.\n"
    f"{datetime_parser.get_format_instructions()}"
)
human_prompt = HumanMessagePromptTemplate.from_template(
    "Convert the following date/time description to a datetime:\nDescription: {date_description}\n"
    "Remember to return only the ISO datetime string in the format: YYYY-MM-DDThh:mm:ss.000000Z"
)

# Combine and format prompts
request = ChatPromptTemplate.from_messages([system_prompt, human_prompt]).format_prompt(
    date_description="the first Monday of next month at 9am"
).to_messages()

# Get response and parse
result = model.invoke(request)
parsed_datetime = datetime_parser.parse(result.content)

# Output results
print("Raw:", result.content)
print("Parsed:", parsed_datetime)
print("Formatted:", parsed_datetime.strftime('%A, %B %d, %Y at %I:%M %p'))

Raw: 2025-02-03T09:00:00.000000Z
Parsed: 2025-02-03 09:00:00
Formatted: Monday, February 03, 2025 at 09:00 AM


In [29]:
datetime_parser.get_format_instructions()

"Write a datetime string that matches the following pattern: '%Y-%m-%dT%H:%M:%S.%fZ'.\n\nExamples: 1717-03-05T06:40:39.947296Z, 0909-08-10T19:17:05.870663Z, 1661-11-03T14:49:16.559227Z\n\nReturn ONLY this string, no other words!"